<a href="https://colab.research.google.com/github/shengy90/MSc-Project/blob/master/notebooks/2nd_August_Further_Exploration_of_Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1️⃣ Setup Notebook 💻**


### **Authenticate with BigQuery ☁️**

In [ ]:
!pip install --upgrade google-cloud-bigquery[bqstorage,pandas]
!pip install --upgrade pandas-gbq

Requirement already up-to-date: google-cloud-bigquery[bqstorage,pandas] in /usr/local/lib/python3.6/dist-packages (1.26.1)
Requirement already up-to-date: pandas-gbq in /usr/local/lib/python3.6/dist-packages (0.13.2)


In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [ ]:
%%bigquery --project machine-learning-msc df --use_bqstorage_api
SELECT 
  COUNT(*) as total_rows
FROM `machine-learning-msc.low_carbon_london.household_consumption_daily_agg` 

In [ ]:
df.head()

,total_rows
0,14841792


### **Importing Libraries⏬**

##### Standard Libraries

In [ ]:
!pip install fbprophet
!pip install MiniSom

  Created wheel for MiniSom: filename=MiniSom-2.2.6-cp36-none-any.whl size=8525 sha256=07ddc633d161f778b86776ad0805ba81b81db372ff108602ec5fddb920b09639
  Stored in directory: /root/.cache/pip/wheels/b8/c6/01/330066e36e1f7c826c96f656f9185822cfcdef0591315949ea
Successfully built MiniSom


In [ ]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt 
import random
import datetime as dt

from minisom import MiniSom
from tqdm import tqdm
from datetime import date
from matplotlib.gridspec import GridSpec
from sklearn.decomposition import PCA
from sklearn.cluster import AgglomerativeClustering
 
sns.set()
%matplotlib inline

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
import pandas_gbq
def output_to_bq(forecast, table_id, project_id='machine-learning-msc'):
    pandas_gbq.to_gbq(forecast, table_id, project_id=project_id, if_exists='append')

##### Import Github Repository

In [56]:
%cd /content
!ls

/content
adc.json  mscproj  sample_data


In [57]:
!rm -rf mscproj
!git clone https://github.com/shengy90/MSc-Project mscproj
!git pull
%cd /content/mscproj/
!ls

Cloning into 'mscproj'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 387 (delta 11), reused 13 (delta 6), pack-reused 361
Receiving objects: 100% (387/387), 10.95 MiB | 4.58 MiB/s, done.
Resolving deltas: 100% (209/209), done.
fatal: not a git repository (or any of the parent directories): .git
/content/mscproj
bin	     __init__.py  notebooks  requirements.txt  sql
definitions  Makefile	  README.md  run.py	       src


In [58]:
%reload_ext autoreload 
%autoreload 2 
from src.train_prophet import TrainProphet
from src.train_clusters import TrainClusters
from src.train_clusters import Normaliser

# 2️⃣ **Training Up to 16 Agglomerative Clusters**

### **Downloading Data from BQ**

In [ ]:
%%bigquery --project machine-learning-msc df_test --use_bqstorage_api
WITH stg1 AS (
SELECT 
lcl_id,
IF(acorn_grouped = "Adversity", 1, 0) AS adversity,
IF(acorn_grouped = "Affluent", 1, 0) AS affluent,
IF(acorn_grouped = "Comfortable", 1, 0) AS comfortable,
FORMAT_DATETIME("%B", DATETIME(ts)) AS month_name,
dayofweek,
hhourly_rank,
ROUND(AVG(kwhh),4) AS hh_avg,
ROUND(MAX(kwhh),4) AS hh_max,
ROUND(MIN(kwhh),4) AS hh_min,
ROUND(STDDEV(kwhh),4) AS hh_stddev

FROM `machine-learning-msc.forecasting_20200719.test_set`
WHERE train_test_split = 'test'
AND ts >= '2012-11-01' AND ts < '2013-03-01'

GROUP BY 1,2,3,4,5,6,7
)

SELECT 
*,
ROW_NUMBER() OVER (PARTITION BY lcl_id, month_name ORDER BY dayofweek ASC, hhourly_rank ASC) AS weekly_rank
FROM stg1 
ORDER BY lcl_id, month_name, weekly_rank, hhourly_rank

In [ ]:
%%bigquery --project machine-learning-msc df_train --use_bqstorage_api
WITH stg1 AS (
SELECT 
lcl_id,
IF(acorn_grouped = "Adversity", 1, 0) AS adversity,
IF(acorn_grouped = "Affluent", 1, 0) AS affluent,
IF(acorn_grouped = "Comfortable", 1, 0) AS comfortable,
FORMAT_DATETIME("%B", DATETIME(ts)) AS month_name,
dayofweek,
hhourly_rank,
ROUND(AVG(kwhh),4) AS hh_avg,
ROUND(MAX(kwhh),4) AS hh_max,
ROUND(MIN(kwhh),4) AS hh_min,
ROUND(STDDEV(kwhh),4) AS hh_stddev

FROM `machine-learning-msc.forecasting_20200719.train_set`
WHERE train_test_split = 'train'
AND ts >= '2012-11-01' AND ts < '2013-03-01'

GROUP BY 1,2,3,4,5,6,7
)

SELECT 
*,
ROW_NUMBER() OVER (PARTITION BY lcl_id, month_name ORDER BY dayofweek ASC, hhourly_rank ASC) AS weekly_rank
FROM stg1 
ORDER BY lcl_id, month_name, weekly_rank, hhourly_rank

### **Normalise Dataset**

In [ ]:
value_list = ['hh_avg']
column_list = ['month_name', 'weekly_rank']
normaliser = Normaliser(value_list, column_list)
norm_df_train = normaliser.fit(df_train)
norm_df_test = normaliser.predict(df_test)

### **1. Training Up to 16 Agglomerative Clusters**


In [ ]:
for i in range(16):
    cluster_num = i+1
    print(f"Training {cluster_num} clusters....")
    agglo_cluster = TrainClusters(cluster_type="agglo")
    agglo_cluster.fit(norm_df_train, cluster_num=cluster_num)

    train_pred = agglo_cluster.predict(norm_df_train)
    test_pred = agglo_cluster.predict(norm_df_test)

    train_pred['train_test_split'] = "train"
    test_pred['train_test_split'] = "test"
    
    agglo_results = pd.concat([train_pred[['lcl_id','cluster','train_test_split']], test_pred[['lcl_id','cluster','train_test_split']]])
    agglo_results['cluster'] = agglo_results['cluster'].astype(float)
    agglo_results['num_clusters'] = cluster_num
    agglo_results['cluster_type'] = 'agglo'

    output_to_bq(agglo_results, 'clusters_20200802.agglo_16_clusters')
    print("Upload to BQ completed! 🎉")

Training 1 clusters....


1it [00:03,  3.17s/it]


Upload to BQ completed! 🎉
Training 2 clusters....


1it [00:03,  3.61s/it]


Upload to BQ completed! 🎉
Training 3 clusters....


...

Upload to BQ completed! 🎉
Training 15 clusters....


1it [00:04,  4.08s/it]


Upload to BQ completed! 🎉
Training 16 clusters....


1it [00:03,  3.71s/it]

Upload to BQ completed! 🎉


### **2. Evaluating Clusters**


##### **Downloading Data from BQ**

In [ ]:
%%bigquery --project machine-learning-msc df_train --use_bqstorage_api
SELECT 
train.lcl_id,
train.ts AS ds,
train.kwhh AS y,
weather.air_temperature

FROM `machine-learning-msc.forecasting_20200719.train_set` train 
LEFT JOIN `machine-learning-msc.london_heathrow_hourly_weather_data.london_heathrow_hourly_weather` weather 
  ON TIMESTAMP_TRUNC(weather.ts, HOUR) = TIMESTAMP_TRUNC(train.ts, hour)

WHERE train.ts >= '2012-11-01' AND train.ts < '2013-03-01'
ORDER BY 1,2 ASC

In [ ]:
%%bigquery --project machine-learning-msc df_test --use_bqstorage_api
SELECT 
train.lcl_id,
train.ts AS ds,
train.kwhh AS y,
weather.air_temperature

FROM `machine-learning-msc.forecasting_20200719.test_set` train 
LEFT JOIN `machine-learning-msc.london_heathrow_hourly_weather_data.london_heathrow_hourly_weather` weather 
  ON TIMESTAMP_TRUNC(weather.ts, HOUR) = TIMESTAMP_TRUNC(train.ts, hour)
  

WHERE train.ts >= '2012-11-01' AND train.ts < '2013-03-01'
ORDER BY 1,2 ASC

In [ ]:
df_train['ds'] = df_train['ds'].dt.tz_localize(None) # remove timezones 
df_test['ds'] = df_test['ds'].dt.tz_localize(None) # remove timezones 

print(df_train.shape, df_test.shape)

(15439123, 4) (5758620, 4)


In [ ]:
%%bigquery --project machine-learning-msc clusters --use_bqstorage_api
SELECT * FROM `machine-learning-msc.clusters_20200802.agglo_16_clusters`

In [ ]:
clusters.head()

,lcl_id,cluster,train_test_split,num_clusters,cluster_type
0,MAC000024,0.0,test,3,agglo
1,MAC000034,0.0,test,3,agglo
2,MAC000312,0.0,test,3,agglo
3,MAC000415,0.0,test,3,agglo
4,MAC000557,0.0,test,3,agglo


##### **Utility Functions..**

In [ ]:
def train_clusters(df_train, df_test, test_period="2013-02-01"):
    forecast_dict = {}
    test_global_fc = pd.DataFrame()
    train_global_fc = pd.DataFrame()
    clusters = df_train.groupby('cluster').count().index.to_list()
    for cluster in clusters:
        cluster_dict = {} 
        print(f"\nTraining cluster: {cluster}") 
        print("---------------------------")
        df_train_cluster = df_train.query(f"cluster=={cluster}").copy()
        df_test_cluster = df_test.query(f"cluster=={cluster}").copy()
        model = TrainProphet(test_period)
        model.fit(df_train_cluster)
        model.evaluate_test_global_mape(df_test_cluster)
        cluster_dict['model'] = model 
        forecast_dict[f'cluster_{cluster}']=cluster_dict
        test_global_fc = pd.concat([test_global_fc, model.test_forecast])

        train_forecast = df_train[['cluster','ds','y']].copy()
        train_forecast['max_households'] = df_train['households_num'].max()
        train_forecast = train_forecast.merge(model.forecast[['ds', 'yhat']], left_on='ds', right_on='ds')
        train_forecast['y_global'] = train_forecast['y'] * train_forecast['max_households']
        train_forecast['yhat_global'] = train_forecast['yhat'] * train_forecast['max_households']
        train_global_fc = pd.concat([train_global_fc, train_forecast])

    return forecast_dict, test_global_fc, train_global_fc

In [ ]:
def get_timeseries(df, clusters_df, cluster_type, num_clusters):
    clusters = clusters_df.query(f"cluster_type=='{cluster_type}' and num_clusters=={num_clusters}")
    out_df = df.merge(clusters[['lcl_id','cluster']], left_on='lcl_id', right_on='lcl_id')
    households_num = pd.DataFrame(out_df.groupby('cluster')['lcl_id'].nunique())
    households_num.rename(columns={'lcl_id':'households_num'}, inplace=True)

    timeseries = out_df.groupby(['cluster','ds']).mean().reset_index()
    timeseries = timeseries.merge(households_num, left_on='cluster', right_on='cluster')
    return timeseries

In [ ]:
def train_cluster_forecast(df_train, df_test, cluster_type, clusters, cluster_list):
    results_dict = {}
    
    for cluster in cluster_list:
        cluster_forecast_dict = {}
        print(f"\n ----------------------------------")
        print(f"|Total number of clusters: {cluster}...   |")
        print(f" ----------------------------------")

        train = get_timeseries(df_train, clusters, cluster_type=cluster_type, num_clusters=cluster)
        test = get_timeseries(df_test, clusters, cluster_type=cluster_type, num_clusters=cluster)

        model_dict, global_test, global_train = train_clusters(train, test)

        cluster_forecast_dict['model'] = model_dict
        cluster_forecast_dict['global_test'] = global_test
        cluster_forecast_dict['global_train'] = global_train 

        results_dict[f"num_clusters_{cluster}"] = cluster_forecast_dict 

    return results_dict

#####**Training Forecasts**

In [ ]:
cluster_list = [2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]
agglo_results = train_cluster_forecast(df_train, df_test, 'agglo', clusters, cluster_list)


 ----------------------------------
|Total number of clusters: 2...   |
 ----------------------------------

Training cluster: 0.0
---------------------------
Training Mean Absolute Percentage Error: 7.790572916666669
Test Mean Absolute Percentage Error: 8.23

Training cluster: 1.0
---------------------------
Training Mean Absolute Percentage Error: 8.77916666666666
Test Mean Absolute Percentage Error: 29.25

 ----------------------------------
|Total number of clusters: 3...   |
 ----------------------------------

Training cluster: 0.0
---------------------------
Training Mean Absolute Percentage Error: 8.569873511904763
Test Mean Absolute Percentage Error: 64.62

Training cluster: 1.0
---------------------------
Training Mean Absolute Percentage Error: 8.77916666666666
Test Mean Absolute Percentage Error: 156.81

Training cluster: 2.0
---------------------------
Training Mean Absolute Percentage Error: 7.447537202380951
Test Mean Absolute Percentage Error: 8.110000000000001

 -----

##### **Evaluate Forecast**

In [ ]:
train_global_results = agglo_results['num_clusters_2']['global_train'].groupby('ds')[['y_global','yhat_global']].sum()
test_global_results = agglo_results['num_clusters_2']['global_test'].groupby('ds')[['y_global','yhat_global']].sum()

train_global_mape = np.round(np.mean(np.abs(train_global_results['yhat_global']/train_global_results['y_global']-1)),4)*100
test_global_mape = np.round(np.mean(np.abs(test_global_results['yhat_global']/test_global_results['y_global']-1)),4)*100

print(train_global_mape, test_global_mape)

6.660000000000001 9.180000000000001


In [ ]:
def evaluate_results(results_dict):
    for num_clusters in results_dict.keys():
        train_results = results_dict[num_clusters]['global_train'].groupby('ds')[['y_global','yhat_global']].sum()
        test_results = results_dict[num_clusters]['global_test'].groupby('ds')[['y_global', 'yhat_global']].sum() 

        train_global_mape = np.round(np.mean(np.abs(train_results['yhat_global']/train_results['y_global']-1)),4)*100
        test_global_mape = np.round(np.mean(np.abs(test_results['yhat_global']/test_results['y_global']-1)),4)*100
        results_dict[num_clusters]['train_global_mape'] = train_global_mape
        results_dict[num_clusters]['test_global_mape'] = test_global_mape

        print(f"Number of Clusters = {num_clusters}: Train Global MAPE: {train_global_mape}. Test Global MAPE: {test_global_mape}")

In [ ]:
evaluate_results(agglo_results)

Number of Clusters = num_clusters_2: Train Global MAPE: 6.660000000000001. Test Global MAPE: 9.180000000000001
Number of Clusters = num_clusters_3: Train Global MAPE: 6.569999999999999. Test Global MAPE: 71.78
Number of Clusters = num_clusters_4: Train Global MAPE: 12.22. Test Global MAPE: 74.11
Number of Clusters = num_clusters_5: Train Global MAPE: 9.27. Test Global MAPE: 77.21000000000001
Number of Clusters = num_clusters_6: Train Global MAPE: 8.64. Test Global MAPE: 58.13
Number of Clusters = num_clusters_7: Train Global MAPE: 9.06. Test Global MAPE: 57.52
Number of Clusters = num_clusters_8: Train Global MAPE: 8.35. Test Global MAPE: 34.02
Number of Clusters = num_clusters_9: Train Global MAPE: 10.0. Test Global MAPE: 34.06
Number of Clusters = num_clusters_10: Train Global MAPE: 9.84. Test Global MAPE: 130.70999999999998
Number of Clusters = num_clusters_11: Train Global MAPE: 9.45. Test Global MAPE: 174.34
Number of Clusters = num_clusters_12: Train Global MAPE: 9.1. Test Global

### **3. Saving results to BigQuery**

In [ ]:
for cluster in agglo_results:
    global_train = agglo_results[cluster]['global_train']
    global_test = agglo_results[cluster]['global_test']

    global_train['cluster_type'] = "agglo"
    global_train['num_clusters'] = cluster 
    global_test['cluster_type'] = "agglo"
    global_test['num_clusters'] = cluster

    output_to_bq(global_train, table_id='clusters_20200802.agglo_16_clusters_train_results')
    output_to_bq(global_test, table_id='clusters_20200802.agglo_16_clusters_test_results')
    print(f"{cluster} results uploaded to BQ!")

23040 out of 23040 rows loaded.
1it [00:05,  5.07s/it]
2688 out of 2688 rows loaded.
1it [00:02,  2.78s/it]
0it [00:00, ?it/s]

num_clusters_2 results uploaded to BQ!


51840 out of 51840 rows loaded.
1it [00:06,  6.28s/it]
4032 out of 4032 rows loaded.
1it [00:05,  5.96s/it]
0it [00:00, ?it/s]

num_clusters_3 results uploaded to BQ!


92160 out of 92160 rows loaded.
1it [00:07,  7.69s/it]
5376 out of 5376 rows loaded.
1it [00:04,  4.40s/it]
0it [00:00, ?it/s]

...

num_clusters_14 results uploaded to BQ!


1296000 out of 1296000 rows loaded.
1it [00:55, 55.39s/it]
20160 out of 20160 rows loaded.
1it [00:03,  3.57s/it]
0it [00:00, ?it/s]

num_clusters_15 results uploaded to BQ!


1474560 out of 1474560 rows loaded.
1it [01:07, 67.60s/it]
21504 out of 21504 rows loaded.
1it [00:02,  2.78s/it]

num_clusters_16 results uploaded to BQ!


# 3️⃣ **SOM clusters on a different Train/Test Split**

### **1. Generating New Train/Test Split**

In [ ]:
%%bigquery --project machine-learning-msc lcl_ids --use_bqstorage_api
SELECT 
DISTINCT lcl_id, 'test' AS train_test_split, 'original_split' AS random_state
FROM `machine-learning-msc.forecasting_20200719.test_set`
UNION ALL 
SELECT 
DISTINCT lcl_id, 'train' AS train_test_split, 'original_split' AS random_state
FROM `machine-learning-msc.forecasting_20200719.train_set`

In [ ]:
from sklearn.model_selection import train_test_split
train_state_100, test_state_100 = train_test_split(lcl_ids, test_size=0.2716653083, random_state=100)
train_state_500, test_state_500 = train_test_split(lcl_ids, test_size=0.2716653083, random_state=500)

In [ ]:
def create_train_test_set(train_df, test_df, random_state_str):
    train = train_df.copy()
    test = test_df.copy()

    train['train_test_split'] = 'train'
    train['random_state'] = random_state_str
    test['train_test_split'] = 'test'
    test['random_state'] = random_state_str

    all = pd.concat([train, test])
    assert len(all) == 3681

    output_to_bq(all, table_id='households.train_test_split')
    print("Data uploaded to BigQuery! 🏄")
    return all

In [ ]:
create_train_test_set(train_state_100, test_state_100, '100')
create_train_test_set(train_state_500, test_state_500, '500')

1it [00:04,  4.24s/it]


Data uploaded to BigQuery! 🏄


1it [00:03,  3.68s/it]

Data uploaded to BigQuery! 🏄


,lcl_id,train_test_split,random_state
1353,MAC005213,train,500
3230,MAC004168,train,500
369,MAC001630,train,500
914,MAC002033,train,500
848,MAC002600,train,500
...,...,...,...
1375,MAC005041,test,500
3550,MAC002527,test,500
1127,MAC000264,test,500
2964,MAC002423,test,500


In [ ]:
output_to_bq(lcl_ids, table_id='households.train_test_split')

1it [00:04,  4.62s/it]


### **2. Training and Evaluating SOM Clusters**

##### **Downloading Data from BQ**

In [ ]:
%%bigquery --project machine-learning-msc df_100 --use_bqstorage_api
WITH 
raw_data AS (
    SELECT * FROM `machine-learning-msc.forecasting_20200719.train_set` data 
    UNION ALL 
    SELECT * FROM `machine-learning-msc.forecasting_20200719.test_set` data 
    ),

stg1 AS (
    SELECT 
    data.lcl_id,
    split.train_test_split,
    FORMAT_DATETIME("%B", DATETIME(data.ts)) AS month_name,
    data.dayofweek,
    data.hhourly_rank,
    ROUND(AVG(data.kwhh),4) AS hh_avg

    FROM raw_data data

    INNER JOIN `machine-learning-msc.households.train_test_split` split 
        ON split.lcl_id = data.lcl_id 

    WHERE data.ts >= '2012-11-01' AND data.ts < '2013-03-01'
    AND split.random_state = '100'

    GROUP BY 1,2,3,4,5
    )

SELECT 
*,
ROW_NUMBER() OVER (PARTITION BY lcl_id, month_name ORDER BY dayofweek ASC, hhourly_rank ASC) AS weekly_rank
FROM stg1 
ORDER BY lcl_id, month_name, weekly_rank, hhourly_rank

In [ ]:
%%bigquery --project machine-learning-msc df_500 --use_bqstorage_api
WITH 
raw_data AS (
    SELECT * FROM `machine-learning-msc.forecasting_20200719.train_set` data 
    UNION ALL 
    SELECT * FROM `machine-learning-msc.forecasting_20200719.test_set` data 
    ),

stg1 AS (
    SELECT 
    data.lcl_id,
    split.train_test_split,
    FORMAT_DATETIME("%B", DATETIME(data.ts)) AS month_name,
    data.dayofweek,
    data.hhourly_rank,
    ROUND(AVG(data.kwhh),4) AS hh_avg

    FROM raw_data data

    INNER JOIN `machine-learning-msc.households.train_test_split` split 
        ON split.lcl_id = data.lcl_id 

    WHERE data.ts >= '2012-11-01' AND data.ts < '2013-03-01'
    AND split.random_state = '500'

    GROUP BY 1,2,3,4,5
    )

SELECT 
*,
ROW_NUMBER() OVER (PARTITION BY lcl_id, month_name ORDER BY dayofweek ASC, hhourly_rank ASC) AS weekly_rank
FROM stg1 
ORDER BY lcl_id, month_name, weekly_rank, hhourly_rank

In [ ]:
df_100_train = df_100.query('train_test_split=="train"')
df_100_test = df_100.query('train_test_split=="test"')

df_500_train = df_500.query('train_test_split=="train"')
df_500_test = df_500.query('train_test_split=="test"')

In [ ]:
df_100_train['lcl_id'].nunique(), df_100_test['lcl_id'].nunique(), df_500_train['lcl_id'].nunique(), df_500_test['lcl_id'].nunique()

(2681, 1000, 2681, 1000)

##### **Normalising Data Set**

In [ ]:
value_list = ['hh_avg']
column_list = ['month_name', 'weekly_rank']

normaliser = Normaliser(value_list, column_list)
norm_df_100_train = normaliser.fit(df_100_train)
norm_df_100_test = normaliser.predict(df_100_test)
norm_df_500_train = normaliser.fit(df_500_train)
norm_df_500_test = normaliser.predict(df_500_test)

##### **Training SOM Clusters**

In [ ]:
def train_clusters(random_state_str, cluster_num, train_df, test_df):
    som = TrainClusters(cluster_type="som")
    som.fit(train_df, cluster_num=cluster_num, sigma=0.1, learning_rate=0.1)
    train_pred = som.predict(train_df)
    test_pred = som.predict(test_df)

    train_pred['train_test_split'] = 'train'
    test_pred['train_test_split'] = 'test'

    results = pd.concat([train_pred[['lcl_id','cluster','train_test_split']],
                         test_pred[['lcl_id','cluster','train_test_split']]])
    
    results['num_clusters'] = cluster_num
    results['random_state'] = random_state_str
    results['cluster_type'] = "som"
    results['date'] = "2020-08-02"

    output_to_bq(results, 'clusters.clusters')
    print("Results uploaded to BQ! 🏵")

In [ ]:
train_clusters("100", 5, norm_df_100_train, norm_df_100_test)
train_clusters("500", 5, norm_df_500_train, norm_df_500_test)

 [ 100000 / 100000 ] 100% - 0:00:00 left 
 quantization error: 19.462641192440024


2681it [00:01, 2143.68it/s]
1000it [00:00, 2093.45it/s]
1it [00:06,  6.15s/it]


Results uploaded to BQ! 🏵
 [ 100000 / 100000 ] 100% - 0:00:00 left 

0it [00:00, ?it/s]


 quantization error: 19.391452420983683


2681it [00:01, 2201.93it/s]
1000it [00:00, 2191.30it/s]
1it [00:10, 10.47s/it]

Results uploaded to BQ! 🏵


In [ ]:
%%bigquery --project machine-learning-msc original_clusters --use_bqstorage_api

SELECT 
lcl_id,
cluster,
train_test_split,
num_clusters,
"original_split" AS random_state,
cluster_type,
"2020-07-30" AS date
FROM `machine-learning-msc.clusters_20200739.clusters`
WHERE cluster_type = 'som' AND num_clusters = 5

In [ ]:
original_clusters.dtypes

lcl_id               object
cluster             float64
train_test_split     object
num_clusters          int64
random_state         object
cluster_type         object
date                 object
dtype: object

In [ ]:
output_to_bq(original_clusters, 'clusters.clusters')

3681 out of 3681 rows loaded.
1it [00:05,  5.13s/it]


##### **Evaluating Clusters: Download Timeseries From BQ**

In [ ]:
%%bigquery --project machine-learning-msc df_time_series --use_bqstorage_api

WITH all_data AS (
    SELECT * FROM `machine-learning-msc.forecasting_20200719.test_set`
    UNION ALL 
    SELECT * FROM `machine-learning-msc.forecasting_20200719.train_set`   
    )

SELECT 
data.lcl_id,
data.ts AS ds,
data.kwhh AS y,
weather.air_temperature

FROM all_data data
LEFT JOIN `machine-learning-msc.london_heathrow_hourly_weather_data.london_heathrow_hourly_weather` weather 
  ON TIMESTAMP_TRUNC(weather.ts, HOUR) = TIMESTAMP_TRUNC(data.ts, hour)
WHERE data.ts >= '2012-11-01' AND data.ts < '2013-03-01'
ORDER BY 1,2 ASC

In [ ]:
df_time_series['ds'] = df_time_series['ds'].dt.tz_localize(None) # remove timezones

In [ ]:
%%bigquery --project machine-learning-msc df_clusters_random_state_100 --use_bqstorage_api
SELECT 
* 
FROM `machine-learning-msc.clusters.clusters` 
WHERE random_state = '100' AND date = "2020-08-02"

In [ ]:
%%bigquery --project machine-learning-msc df_clusters_random_state_500 --use_bqstorage_api
SELECT 
* 
FROM `machine-learning-msc.clusters.clusters` 
WHERE random_state = '500' AND date = "2020-08-02"

##### **Evaluating Clusters: Training Forecasts**

In [ ]:
def train_clusters(df_train, df_test, test_period="2013-02-01"):
    forecast_dict = {}
    test_global_fc = pd.DataFrame()
    train_global_fc = pd.DataFrame()
    clusters = df_train.groupby('cluster').count().index.to_list()
    for cluster in clusters:
        cluster_dict = {} 
        print(f"\nTraining cluster: {cluster}") 
        print("---------------------------")
        df_train_cluster = df_train.query(f"cluster=={cluster}").copy()
        df_test_cluster = df_test.query(f"cluster=={cluster}").copy()
        model = TrainProphet(test_period)
        model.fit(df_train_cluster)
        model.evaluate_test_global_mape(df_test_cluster)
        cluster_dict['model'] = model 
        forecast_dict[f'cluster_{cluster}']=cluster_dict
        test_global_fc = pd.concat([test_global_fc, model.test_forecast])

        train_forecast = df_train[['cluster','ds','y']].copy()
        train_forecast['max_households'] = df_train['households_num'].max()
        train_forecast = train_forecast.merge(model.forecast[['ds', 'yhat']], left_on='ds', right_on='ds')
        train_forecast['y_global'] = train_forecast['y'] * train_forecast['max_households']
        train_forecast['yhat_global'] = train_forecast['yhat'] * train_forecast['max_households']
        train_global_fc = pd.concat([train_global_fc, train_forecast])

    return forecast_dict, test_global_fc, train_global_fc

In [ ]:
def get_timeseries(df, clusters_df, cluster_type, num_clusters):
    clusters = clusters_df.query(f"cluster_type=='{cluster_type}' and num_clusters=={num_clusters}")
    out_df = df.merge(clusters[['lcl_id','cluster']], left_on='lcl_id', right_on='lcl_id')

    assert out_df['lcl_id'].nunique() == clusters['lcl_id'].nunique()
    
    households_num = pd.DataFrame(out_df.groupby('cluster')['lcl_id'].nunique())
    households_num.rename(columns={'lcl_id':'households_num'}, inplace=True)

    timeseries = out_df.groupby(['cluster','ds']).mean().reset_index()
    timeseries = timeseries.merge(households_num, left_on='cluster', right_on='cluster')
    return timeseries

In [ ]:
def evaluate_results(global_train, global_test):
    train = global_train.groupby('ds')[['y_global', 'yhat_global']].sum()
    test = global_test.groupby('ds')[['y_global', 'yhat_global']].sum()

    train_mape = np.round(np.mean(np.abs(train['yhat_global']/train['y_global']-1)),4)*100
    test_mape = np.round(np.mean(np.abs(test['yhat_global']/test['y_global']-1)),4)*100

    print(f"Train global MAPE: {train_mape}. Test global MAPE: {test_mape}.")
    return train_mape, test_mape

##### Random State 100 Train Test Splits

In [ ]:
train_100 = get_timeseries(df_time_series, df_clusters_random_state_100.query("train_test_split=='train'"), cluster_type="som", num_clusters=5)
test_100 = get_timeseries(df_time_series, df_clusters_random_state_100.query("train_test_split=='test'"), cluster_type="som", num_clusters=5)

In [ ]:
model_dict, global_test_100, global_train_100 = train_clusters(train_100, test_100)


Training cluster: 0.0
---------------------------
Training Mean Absolute Percentage Error: 8.749568452380938
Test Mean Absolute Percentage Error: 10.67

Training cluster: 1.0
---------------------------
Training Mean Absolute Percentage Error: 9.242745535714276
Test Mean Absolute Percentage Error: 16.1

Training cluster: 2.0
---------------------------
Training Mean Absolute Percentage Error: 6.979308035714286
Test Mean Absolute Percentage Error: 7.84

Training cluster: 3.0
---------------------------
Training Mean Absolute Percentage Error: 66.5159077380953
Test Mean Absolute Percentage Error: 74.42999999999999

Training cluster: 4.0
---------------------------
Training Mean Absolute Percentage Error: 8.665863095238118
Test Mean Absolute Percentage Error: 10.61


In [ ]:
evaluate_results(global_train_100, global_test_100)

Train global MAPE: 9.64. Test global MAPE: 7.93.


##### Random State 500 Train Test Splits

In [ ]:
train_500 = get_timeseries(df_time_series, df_clusters_random_state_500.query("train_test_split=='train'"), cluster_type="som", num_clusters=5)
test_500 = get_timeseries(df_time_series, df_clusters_random_state_500.query("train_test_split=='test'"), cluster_type="som", num_clusters=5)
model_dict, global_test_500, global_train_500 = train_clusters(train_500, test_500)


Training cluster: 0.0
---------------------------
Training Mean Absolute Percentage Error: 8.693913690476197
Test Mean Absolute Percentage Error: 11.87

Training cluster: 1.0
---------------------------
Training Mean Absolute Percentage Error: 10.70595982142858
Test Mean Absolute Percentage Error: 16.76

Training cluster: 2.0
---------------------------
Training Mean Absolute Percentage Error: 7.108831845238102
Test Mean Absolute Percentage Error: 8.1

Training cluster: 3.0
---------------------------
Training Mean Absolute Percentage Error: 9.402187500000014
Test Mean Absolute Percentage Error: 9.45

Training cluster: 4.0
---------------------------
Training Mean Absolute Percentage Error: 69.7506994047619
Test Mean Absolute Percentage Error: 50.39


In [ ]:
evaluate_results(global_test_500, global_train_500)

Train global MAPE: 7.3. Test global MAPE: 9.049999999999999.


# 4️⃣ **Using 2012 Winter to predict 2013 winter**

### **⏬ Downloading data from BQ**

##### Train test splits

In [ ]:
%%bigquery --project machine-learning-msc df_train_test_splits --use_bqstorage_api
SELECT * FROM `machine-learning-msc.households.train_test_split` 

##### Timeseries forecast inputs

In [ ]:
%%bigquery --project machine-learning-msc df_2012_timeseries --use_bqstorage_api

WITH all_data AS (
    SELECT * FROM `machine-learning-msc.forecasting_20200719.test_set`
    UNION ALL 
    SELECT * FROM `machine-learning-msc.forecasting_20200719.train_set`   
    )

SELECT
data.lcl_id,
data.ts AS ds,
data.kwhh AS y,
weather.air_temperature

FROM all_data data
LEFT JOIN `machine-learning-msc.london_heathrow_hourly_weather_data.london_heathrow_hourly_weather` weather 
  ON TIMESTAMP_TRUNC(weather.ts, HOUR) = TIMESTAMP_TRUNC(data.ts, hour)
WHERE data.ts >= '2012-11-01' AND data.ts < '2013-03-01'
ORDER BY 1,2 ASC

In [60]:
%%bigquery --project machine-learning-msc df_2013_timeseries --use_bqstorage_api

WITH all_data AS (
    SELECT * FROM `machine-learning-msc.forecasting_20200719.test_set`
    UNION ALL 
    SELECT * FROM `machine-learning-msc.forecasting_20200719.train_set`   
    )

SELECT
data.lcl_id,
data.ts AS ds,
data.kwhh AS y,
weather.air_temperature

FROM all_data data
LEFT JOIN `machine-learning-msc.london_heathrow_hourly_weather_data.london_heathrow_hourly_weather` weather 
  ON TIMESTAMP_ADD(TIMESTAMP_TRUNC(weather.ts, HOUR), INTERVAL 365 DAY) = TIMESTAMP_TRUNC(data.ts, hour)
WHERE data.ts >= '2013-11-01' AND data.ts < '2014-03-01'
ORDER BY 1,2 ASC

In [61]:
df_2012_timeseries['ds'] = df_2012_timeseries['ds'].dt.tz_localize(None) # remove timezones
df_2013_timeseries['ds'] = df_2013_timeseries['ds'].dt.tz_localize(None) # remove timezones

##### Cluster Inputs

In [17]:
%%bigquery --project machine-learning-msc df_2012_aggregated_ts --use_bqstorage_api
WITH 
raw_data AS (
    SELECT * FROM `machine-learning-msc.forecasting_20200719.train_set` data 
    UNION ALL 
    SELECT * FROM `machine-learning-msc.forecasting_20200719.test_set` data 
    ),
stg1 AS (
    SELECT 
    data.lcl_id,
    FORMAT_DATETIME("%B", DATETIME(data.ts)) AS month_name,
    data.dayofweek,
    data.hhourly_rank,
    ROUND(AVG(data.kwhh),4) AS hh_avg

    FROM raw_data data
    WHERE data.ts >= '2012-11-01' AND data.ts < '2013-03-01'
    GROUP BY 1,2,3,4
    )

SELECT 
*,
ROW_NUMBER() OVER (PARTITION BY lcl_id, month_name ORDER BY dayofweek ASC, hhourly_rank ASC) AS weekly_rank
FROM stg1 
ORDER BY lcl_id, month_name, weekly_rank, hhourly_rank

In [18]:
%%bigquery --project machine-learning-msc df_2013_aggregated_ts --use_bqstorage_api
WITH 
raw_data AS (
    SELECT * FROM `machine-learning-msc.forecasting_20200719.train_set` data 
    UNION ALL 
    SELECT * FROM `machine-learning-msc.forecasting_20200719.test_set` data 
    ),
stg1 AS (
    SELECT 
    data.lcl_id,
    FORMAT_DATETIME("%B", DATETIME(data.ts)) AS month_name,
    data.dayofweek,
    data.hhourly_rank,
    ROUND(AVG(data.kwhh),4) AS hh_avg

    FROM raw_data data
    WHERE data.ts >= '2013-11-01' AND data.ts < '2014-03-01'
    GROUP BY 1,2,3,4
    )

SELECT 
*,
ROW_NUMBER() OVER (PARTITION BY lcl_id, month_name ORDER BY dayofweek ASC, hhourly_rank ASC) AS weekly_rank
FROM stg1 
ORDER BY lcl_id, month_name, weekly_rank, hhourly_rank

In [19]:
print(len(df_2013_aggregated_ts), len(df_2012_aggregated_ts), len(df_2013_timeseries), len(df_2012_timeseries))

4841157 4947264 20468887 21197743


### **🧪 Create Train and Test set data**

In [20]:
def get_inputs(random_state, df_train_test_splits, df_2012, df_2013):
    # Train set : 2012 data 
    train_df = df_2012.merge(
        df_train_test_splits.query(f"random_state=='{random_state}' and train_test_split=='train'"),
        left_on='lcl_id',
        right_on='lcl_id',
        how='inner'
        )
    # Test set: 2013 data
    test_df = df_2013.merge(
        df_train_test_splits.query(f"random_state=='{random_state}' and train_test_split=='test'"),
        left_on='lcl_id',
        right_on='lcl_id',
        how='inner'
        )

    assert train_df['lcl_id'].nunique() == 2681
    assert test_df['lcl_id'].nunique() < train_df['lcl_id'].nunique()
    return train_df, test_df

In [94]:
def get_timeseries_inputs(random_state, df_cluster, df_ts):
  
    df_forecast = df_ts.merge(
        df_cluster,
        left_on='lcl_id',
        right_on='lcl_id',
        how='inner'
        )
    
    households_num = pd.DataFrame(df_forecast.groupby('cluster')['lcl_id'].nunique())
    households_num.rename(columns={'lcl_id':'households_num'}, inplace=True)
    households_num.reset_index(inplace=True)

    timeseries = df_forecast.groupby(['cluster','ds']).mean().reset_index()
    timeseries = timeseries.merge(households_num, left_on='cluster', right_on='cluster')

    return timeseries

In [73]:
def evaluate_results(global_train, global_test):
    train = global_train.groupby('ds')[['y_global', 'yhat_global']].sum()
    test = global_test.groupby('ds')[['y_global', 'yhat_global']].sum()

    train_mape = np.round(np.mean(np.abs(train['yhat_global']/train['y_global']-1)),4)*100
    test_mape = np.round(np.mean(np.abs(test['yhat_global']/test['y_global']-1)),4)*100

    print(f"Train global MAPE: {train_mape}. Test global MAPE: {test_mape}.")
    return train_mape, test_mape

### **🏋️‍♂️ Training Clusters**

In [21]:
df_cluster_train, df_cluster_test = get_inputs('original_split', df_train_test_splits, df_2012_aggregated_ts, df_2013_aggregated_ts)

In [22]:
value_list = ['hh_avg']
column_list = ['month_name', 'weekly_rank']

normaliser = Normaliser(value_list, column_list)
df_cluster_train_norm = normaliser.fit(df_cluster_train)
df_cluster_test_norm = normaliser.predict(df_cluster_test)

som = TrainClusters(cluster_type="som")
som.fit(df_cluster_train_norm, cluster_num=5, sigma=0.1, learning_rate=0.1)

 [ 100000 / 100000 ] 100% - 0:00:00 left 
 quantization error: 19.84157462314047


In [23]:
cluster_train_pred = som.predict(df_cluster_train_norm)[['lcl_id', 'cluster']]
cluster_test_pred = som.predict(df_cluster_test_norm)[['lcl_id', 'cluster']]

2681it [00:01, 1970.83it/s]
996it [00:00, 2019.70it/s]


### 🦾 **Evaluating Clusters**

In [66]:
random_state = 'original_split'
df_timeseries_train = get_timeseries_inputs(random_state=random_state,
                                            df_train_test_splits=df_train_test_splits,
                                            df_cluster=cluster_train_pred,
                                            df_ts=df_2012_timeseries)
df_timeseries_test  = get_timeseries_inputs(random_state=random_state,
                                            df_train_test_splits=df_train_test_splits,
                                            df_cluster=cluster_test_pred,
                                            df_ts=df_2013_timeseries)

In [67]:
print(len(df_timeseries_train), len(df_timeseries_test))

28800 28565


In [89]:
df_timeseries_train

,cluster,ds,y,air_temperature,households_num
0,0.0,2012-11-01 00:00:00,1.018714,11.8,42
1,0.0,2012-11-01 00:30:00,1.024810,11.8,42
2,0.0,2012-11-01 01:00:00,0.914690,8.8,42
3,0.0,2012-11-01 01:30:00,0.860905,8.8,42
4,0.0,2012-11-01 02:00:00,0.757500,8.5,42
...,...,...,...,...,...
28795,4.0,2013-02-28 21:30:00,0.187646,4.0,1460
28796,4.0,2013-02-28 22:00:00,0.178670,3.5,1460
28797,4.0,2013-02-28 22:30:00,0.166942,3.5,1460
28798,4.0,2013-02-28 23:00:00,0.153369,2.7,1460


In [92]:
clusters = df_timeseries_train.groupby('cluster').count().index.to_list()

train_global_fc = pd.DataFrame()
test_global_fc = pd.DataFrame()

for cluster in clusters:

    train = df_timeseries_train.query(f"cluster=={cluster}").copy()
    test = df_timeseries_test.query(f"cluster=={cluster}").copy()

    print(f"\nTraining cluster: {cluster}. Number of train households: {train['households_num'].max()}. Number of test households: {test['households_num'].max()}.")

    model = TrainProphet("2013-02-01")
    model.fit(train)
    model.evaluate_test_global_mape(test, test_period="2014-02-01")
    test_global_fc = pd.concat([test_global_fc, model.test_forecast])


    train_forecast = train[['cluster','ds','y']].copy()
    train_forecast['max_households'] = train['households_num'].max()
    train_forecast = train_forecast.merge(model.forecast[['ds', 'yhat']], left_on='ds', right_on='ds')
    train_forecast['y_global'] = train_forecast['y'] * train_forecast['max_households']
    train_forecast['yhat_global'] = train_forecast['yhat'] * train_forecast['max_households']
    train_global_fc = pd.concat([train_global_fc, train_forecast])


Training cluster: 0.0. Number of train households: 42. Number of test households: 40.
Training Mean Absolute Percentage Error: 9.670476190476199
Test Mean Absolute Percentage Error: 125.61

Training cluster: 1.0. Number of train households: 29. Number of test households: 15.
Training Mean Absolute Percentage Error: 78.54380208333343
Test Mean Absolute Percentage Error: 158.38

Training cluster: 2.0. Number of train households: 870. Number of test households: 285.
Training Mean Absolute Percentage Error: 9.401599702380938
Test Mean Absolute Percentage Error: 9.629999999999999

Training cluster: 3.0. Number of train households: 280. Number of test households: 83.
Training Mean Absolute Percentage Error: 8.524538690476184
Test Mean Absolute Percentage Error: 85.1

Training cluster: 4.0. Number of train households: 1460. Number of test households: 573.
Training Mean Absolute Percentage Error: 6.989717261904765
Test Mean Absolute Percentage Error: 40.739999999999995


In [93]:
evaluate_results(train_global_fc, test_global_fc)

Train global MAPE: 6.97. Test global MAPE: 46.68.


(6.97, 46.68)

### 🦿**Evaluating Baseline**

In [127]:
def get_baseline_timeseries_input(random_state, df_2012, df_2013, df_train_test_splits):
    
    train = df_2012.merge(df_train_test_splits.query(f"random_state=='{random_state}' and train_test_split=='train'"),
                           left_on='lcl_id',
                           right_on='lcl_id',
                           how='inner'
                            )
    
    test = df_2013.merge(df_train_test_splits.query(f"random_state=='{random_state}' and train_test_split=='test'"),
                        left_on='lcl_id',
                        right_on='lcl_id',
                        how='inner'
                        )

    train['households_num'] = train['lcl_id'].nunique()
    test['households_num'] = test['lcl_id'].nunique()

    train = train.groupby('ds').mean().reset_index()
    test = test.groupby('ds').mean().reset_index()

    assert np.all(train['households_num']==2681)
    assert np.all(train['households_num']>test['households_num'].max())
    
    return train, test

In [128]:
baseline_train_df, baseline_test_df = get_baseline_timeseries_input('original_split', 
                                                                    df_2012_timeseries,
                                                                    df_2013_timeseries,
                                                                    df_train_test_splits)

In [130]:
baseline_model = TrainProphet("2013-02-01")
baseline_model.fit(baseline_train_df)
baseline_model.evaluate_test_global_mape(baseline_test_df, test_period="2014-02-01")

Training Mean Absolute Percentage Error: 7.438474702380949
Test Mean Absolute Percentage Error: 33.03
